In [3]:
from pycoingecko import CoinGeckoAPI
from pandas import DataFrame

# Initialize the CoinGecko API client
cg = CoinGeckoAPI()

In [7]:
# Get list of categories to find gaming related tokens
categories = DataFrame(cg.get_coins_categories())

categories[categories['id'].str.contains('gaming')]

,id,name,market_cap,market_cap_change_24h,content,top_3_coins_id,top_3_coins,volume_24h,updated_at
43,gaming,Gaming (GameFi),1.595594e+10,-3.685617,"GameFi combines blockchain technology, gaming,...","[immutable-x, floki, gala]",[https://coin-images.coingecko.com/coins/image...,1.988338e+09,2024-10-08T17:35:23.131Z
74,gaming-blockchains,Gaming Blockchains,5.153948e+09,-5.672662,Gaming blockchains refer to blockchain platfor...,"[immutable-x, beam-2, ronin]",[https://coin-images.coingecko.com/coins/image...,1.981683e+08,2024-10-08T17:36:09.700Z
81,gaming-utility-token,Gaming Utility Token,3.208630e+09,-3.479953,Gaming Utility Tokens provide holders with ben...,"[axie-infinity, the-sandbox, apecoin]",[https://coin-images.coingecko.com/coins/image...,3.419102e+08,2024-10-08T17:35:14.668Z
85,gaming-governance-token,Gaming Governance Token,2.854059e+09,-3.536201,Gaming Governance Token gives holders the righ...,"[axie-infinity, apecoin, stepn]",[https://coin-images.coingecko.com/coins/image...,3.139811e+08,2024-10-08T17:36:10.835Z
106,gaming-platform,Gaming Platform,1.450865e+09,-2.385799,"Gaming platform allows users to play, download...","[gala, hooked-protocol, nakamoto-games]",[https://coin-images.coingecko.com/coins/image...,2.629321e+08,2024-10-08T17:36:10.890Z
217,on-chain-gaming,On-chain Gaming,3.534825e+07,-1.440917,On-chain games refer to games where all compon...,"[shrapnel-2, lords, cryptoforce]",[https://coin-images.coingecko.com/coins/image...,2.490602e+06,2024-10-08T17:36:16.455Z


In [9]:
low_market_cap_threshold=100000000

min_commits_4weeks=10

# Fetch market data for cryptocurrencies (up to 250 per page)
data = DataFrame(cg.get_coins_markets(vs_currency='usd', order='market_cap_desc', per_page=250, page=1, category='gaming'))
    
data.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,immutable-x,imx,Immutable,https://coin-images.coingecko.com/coins/images...,1.430000,2302587607,43,2868732851,78345859.0,1.570000,...,2.000000e+09,2.000000e+09,9.520000,-84.93069,2021-11-26T01:03:01.536Z,3.780550e-01,279.46581,2022-12-31T07:36:37.649Z,None,2024-10-08T17:39:38.836Z
1,floki,floki,FLOKI,https://coin-images.coingecko.com/coins/images...,0.000135,1308953574,64,1350910763,330477325.0,0.000144,...,1.000000e+13,1.000000e+13,0.000345,-60.87751,2024-06-05T07:25:59.137Z,8.428000e-08,160024.71429,2021-07-06T01:11:20.438Z,None,2024-10-08T17:39:37.577Z
2,gala,gala,GALA,https://coin-images.coingecko.com/coins/images...,0.020535,812254552,92,812254550,129733378.0,0.022242,...,3.950753e+10,5.000000e+10,0.824837,-97.50605,2021-11-26T01:03:48.731Z,1.347500e-04,15166.11849,2020-12-28T08:46:48.367Z,None,2024-10-08T17:39:41.554Z
3,notcoin,not,Notcoin,https://coin-images.coingecko.com/coins/images...,0.007725,792678649,96,792678649,199532748.0,0.008302,...,1.024744e+11,NaN,0.028361,-72.66021,2024-06-02T18:00:38.587Z,4.610570e-03,68.17810,2024-05-24T07:12:14.147Z,None,2024-10-08T17:39:39.074Z
4,beam-2,beam,Beam,https://coin-images.coingecko.com/coins/images...,0.014925,771067676,97,931878407,32402872.0,0.016178,...,6.243401e+10,6.243401e+10,0.044163,-66.19972,2024-03-10T10:40:22.381Z,4.338300e-03,244.08002,2023-10-29T08:20:14.064Z,None,2024-10-08T17:39:42.572Z


In [10]:
# Filter cryptocurrencies with market cap below the threshold
data = data[data['market_cap'] < low_market_cap_threshold]

# Columns to store activity info
data['github_forks'] = 0
data['github_stars'] = 0
data['github_subscribers'] = 0
data['commits_last_4_weeks'] = 0

# Iterate over each cryptocurrency and fetch developer data
for i, row in data.iterrows():
    coin_id = row['id']
    try:
        # Fetch detailed data for each cryptocurrency
        coin_details = cg.get_coin_by_id(coin_id)
        
        # Get developer data
        developer_data = coin_details.get('developer_data', {})
        github_repos = coin_details.get('links', {}).get('repos_url', {}).get('github', [])
        
        # Update developer activity columns
        data.at[i, 'github_forks'] = developer_data.get('forks', 0)
        data.at[i, 'github_stars'] = developer_data.get('stars', 0)
        data.at[i, 'github_subscribers'] = developer_data.get('subscribers', 0)
        data.at[i, 'commits_last_4_weeks'] = developer_data.get('commit_count_4_weeks', 0)
    except Exception as e:
        print(f"Error fetching data for {coin_id}: {e}")

# Filter for projects with consistent GitHub activity (e.g., commits > min_commits_4weeks)
active_projects_df = data[data['commits_last_4_weeks'] > min_commits_4weeks]

# Select relevant columns to display
active_projects_df = active_projects_df[['id', 'name', 'symbol', 'market_cap', 'commits_last_4_weeks', 
                                            'github_forks', 'github_stars', 'github_subscribers']]

active_projects_df

,id,name,symbol,market_cap,commits_last_4_weeks,github_forks,github_stars,github_subscribers
173,chimaera,XAYA,wchi,2104236,89,56,100,17
